In [1]:
import pycbc.noise
import pycbc.psd
import pycbc.filter
from pycbc.types import timeseries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy import signal
import matplotlib.pyplot as plt
from numpy.random import uniform, randint
import pylab
from pycbc.filter import sigma
from pycbc.waveform import get_td_waveform
from pycbc.psd import interpolate
from pycbc.types.timeseries import load_timeseries

#import false signal functions
from ipynb.fs.full.false_signals import random_false_sig, flip_gw

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


# Generate pure noise / signals

In [4]:
# Generate 1 seconds of noise at 4096 Hz, randomized
# argument:
# sig: timeseries
# snr: desired signal-noise ratio
# output noise is scaled to desired snr

def noise(sig,snr):
    # psd
    flow = 20.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)
    
    # un-scaled noise
    delta_t = 1.0 / 4096
    tsamples = int(1.0 / delta_t)
    ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)
    
    # Scale
    new_psd = pycbc.psd.estimate.interpolate(psd, sig.delta_f)  # First interpolate psd to desire delta_f
    current_snr = sigma(sig, psd=new_psd, low_frequency_cutoff=20.0)
    c = current_snr/snr
    scaled_ts = c * ts

    return scaled_ts

In [5]:
# Generate gravitational waveform with specified mass parameters.
# Other arguments fixed
# return the plus polarization

# mass range does not need to be restricted any more.
# for signal length < 1, it is shifted. 
# for signal length > 1, the waveform is cropped on the left to keep the right-hand side with the merger moement.
# For the ease of merging, the start_time is reset to 0 and total length is 1

def gw(m1,m2):
    hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                          mass1=m1,    
                          mass2=m2,
                          delta_t=1.0/4096,
                          f_lower=20,
                          distance=100)
    
    # randomly shift
    # To fit the whole waveform in the 1 second window, need to specify a cyclic shift range. 
    # 0.08 act as buffer since signal still oscillates a little after merging.
    sigtime = abs(hp.start_time) + 0.08
    
    if sigtime < 1:
        # resize to window of 1 second
        hp.resize(4096)
        shift_range = 1 - sigtime
        shifted = hp.cyclic_time_shift(uniform(0.0,shift_range))
    else:
        totaltime = hp.duration
        hp.start_time = 0
        hp = hp.crop(0, totaltime - sigtime)
        shifted = hp.crop(hp.duration-1,0)
    
    # reset start time
    shifted.start_time = 0
    
    return shifted

In [11]:
# scaling factor. Originally noise contained within around 1*10^-21
# like noise, but noise is mainly an internal function

def empty_gaussian(c):
    # psd
    flow = 20.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)
    
    # un-scaled noise
    delta_t = 1.0 / 4096
    tsamples = int(1.0 / delta_t)
    ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)
    
    # Scale
    scaled_ts = c * ts

    return scaled_ts

# Generate signals burried in noise with a specified SNR

In [7]:
def burried_gw(m1,m2,snr):
    sig = gw(m1,m2)
    noi = noise(sig,snr)
    
    return noi + sig

In [8]:
def burried_false(mag,snr):
    fal = random_false_sig(mag)
    fal.resize(4096)
    noi = noise(fal,snr)
    
    return noi + fal

In [9]:
def burried_flip(m1,m2,snr):
    hp = gw(m1,m2)
    flipped = flip_gw(hp)
    noi = noise(hp,snr)
    
    return noi + flipped

In [9]:

'''
# test
test1 = burried_gw(10,15,1000)
test2 = burried_flip(30,35,1000)
test3 = burried_false(-21,1000)

# plot
pylab.plot(test1.sample_times, test1)
pylab.show()
# plot
pylab.plot(test2.sample_times, test2)
pylab.show()
# plot
pylab.plot(test3.sample_times, test3)
pylab.show()

visualize(test1)
visualize(test3)
'''


'\n# test\ntest1 = burried_gw(10,15,1000)\ntest2 = burried_flip(30,35,1000)\ntest3 = burried_false(-21,1000)\n\n# plot\npylab.plot(test1.sample_times, test1)\npylab.show()\n# plot\npylab.plot(test2.sample_times, test2)\npylab.show()\n# plot\npylab.plot(test3.sample_times, test3)\npylab.show()\n\nvisualize(test1)\nvisualize(test3)\n'

# Partial signals burried in noise
for false, it is randomly selected and randomly burried
for gw, certain percentage on right side is kept for practical purposes

In [12]:
# return partial_burried_false_signals 
# magnitude and snr same as burried_false
# percentage: keep 0 to 1 of the waveform, randomly chosen from anypart of the original waveform

def partial_burried_false(mag,snr,percentage):
    fal = random_false_sig(mag)
    fal.resize(4096)
    noi = noise(fal,snr)
    
    # left + right = 1 - percentage
    left = uniform(0,1-percentage)
    right = (1-percentage)-left
    fal = fal.crop(left, right)
    
    # complete to 4096 data
    zeros = 4096 - len(fal)
    left_zeros = randint(zeros+1)
    right_zeros = zeros - left_zeros 
    fal.append_zeros(right_zeros)
    fal.prepend_zeros(left_zeros)
    
    fal.start_time = 0
    
    return fal+noi

In [14]:
# return partial_burried_signals 
# The partial signal generated by cropping off left side and append zeros on right
# In this way, the merger moement is included. 
# The purpose is that the net can detect the gw as long as merger moment included and a signal of 50%-100% remains.
# percentage means the part kept

def partial_burried_gw(m1,m2,snr,percentage):
    hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                          mass1=m1,    
                          mass2=m2,
                          delta_t=1.0/4096,
                          f_lower=20,
                          distance=100)

    # crop to only keep signal
    sigtime = abs(hp.start_time) + 0.08
    totaltime = hp.duration
    hp.start_time = 0
    hp = hp.crop(0, totaltime - sigtime)
    
    # if really long only keep 1 sec
    if sigtime > 1:
        hp = hp.crop(hp.duration-1,0)
        left = 1-percentage
    else:
        left = (1-percentage) * sigtime
    # now keep only certain percentage from right end side
    hp.start_time = 0
    
    hp = hp.crop(left, 0)
    
    # complete to 4096 data
    zeros = 4096 - len(hp)
    hp.append_zeros(zeros)
    
    # burry in noise
    noi = noise(hp,snr)    
    hp.start_time = 0
        
    return noi + hp

In [15]:
'''
# test the functions

A  = partial_burried_false(-21,600,0.3)
pylab.plot(A.sample_times, A)
pylab.show()

B  = partial_burried_gw(100,50,600,0.5)
pylab.plot(B.sample_times, B)
pylab.show()

C = burried_gw(100,50,600)
pylab.plot(C.sample_times, C)
pylab.show()

'''


'\n# test the functions\n\nA  = partial_burried_false(-21,600,0.3)\npylab.plot(A.sample_times, A)\npylab.show()\n\nB  = partial_burried_gw(100,50,600,0.5)\npylab.plot(B.sample_times, B)\npylab.show()\n\nC = burried_gw(100,50,600)\npylab.plot(C.sample_times, C)\npylab.show()\n\n'

# Other functions

In [16]:
# define a function to transform numpy array to pycbc timeseries.

def convert(w):
    # get sample times
    hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                          mass1=20,    
                          mass2=20,
                          delta_t=1.0/4096,
                          f_lower=20,
                          distance=100)
    hp.resize(4096)
    hp.start_time = 0
    times = hp.sample_times
    
    wform = pd.Series(w, index =times)
    
    # get pycbc series!
    f_path = 'temporary.txt'
    f= open(f_path,"w+").close()
    wform.to_csv(f_path,sep=' ')
    wform = load_timeseries(f_path)
    
    return wform

In [17]:
# Output one True after every k False, total length n
# For data selection purpose
def bol(k,n):
    bol = []
    for i in np.arange(n/(k)):
        for l in np.arange(k-1):
            bol.append(False)
        bol.append(True)
    return bol

In [18]:
# A crude visualization because not sure how to tune parameters for qtransform.
# Show the plot

def visualize(wave):
    times, freqs, power = wave.qtransform(.001, logfsteps=100,
                                                qrange=(4, 4),
                                                frange=(20, 512),)
    pylab.figure(figsize=[15, 3])
    pylab.pcolormesh(times, freqs, power**0.5)
    pylab.yscale('log')
    pylab.show()

# Modified false signal functions for 2B

In [19]:
# return partially burried gausspulse signals as coded for 2B
# percentage: keep 0 to 1 of the waveform, randomly chosen from anypart of the original waveform

def partial_burried_2B(wf,snr,percentage):
    fal = wf
    fal.resize(4096)
    noi = noise(fal,snr)
    
    # left + right = 1 - percentage
    left = uniform(0,1-percentage)
    right = (1-percentage)-left
    fal = fal.crop(left, right)
    
    # complete to 4096 data
    zeros = 4096 - len(fal)
    left_zeros = randint(zeros+1)
    right_zeros = zeros - left_zeros 
    fal.append_zeros(right_zeros)
    fal.prepend_zeros(left_zeros)
    
    fal.start_time = 0
    
    return fal+noi


In [20]:
'''

# test
gww = burried_gw(20, 60, 20)
visualize(gww)
'''


'\n\n# test\ngww = burried_gw(20, 60, 20)\nvisualize(gww)\n'